In [30]:
import random
import re
import string
from collections import Counter
import math

class Wordle(object):

    def __init__(self):
        self._answers = self.readfiles('2315_possible_answers.txt')
        self._allowed = self.readfiles('12972_allowed_words.txt')
        self._allowedWordEntropy = {line.split()[0] : float(line.split()[1])  for line in self.readfiles('12972_allowed_words_entroy.txt')}
        self._passed = self._allowed
        #self.updateEnropy()

    # ---    Word Guessed  -----------   
        #self._wordleWord = random.choice(self._answers)
        #print(self._wordleWord.upper())

    #----   Reading Files -------

    def readfiles(self, file , entropy = False):
        with open(file,'r') as fh:
            lines = fh.readlines()
        return [line.rstrip('\n') for line in lines]

    #----- Green Gray or Yellow
    def isMatched(self,text, pattern):
        if re.search(pattern, text):
            return True
        return False
    
    def gray(self, grayset, allowed = None):
        if allowed is None:
            allowed = self._passed

        if grayset == "":
            return allowed
        
        pattern = re.compile('[{0}]'.format(grayset))
        passed_ = []
        for word in allowed:
            if not self.isMatched(word, pattern):
                passed_.append(word)
        
        return passed_


    def green(self,greenList,allowed):
        passed_ = []       
        if greenList == []:
            return allowed
        else:
            greenList = list(greenList)
            char , position = greenList[0]
            for word in allowed:
                if word[position] == char:
                    passed_.append(word)
            return self.green(greenList[1:],passed_)
    
    def notGreen(self,greenList,allowed):
        passed_ = []       
        if greenList == []:
            return allowed
        else:
            char , position = greenList[0]
            for word in allowed:
                if word[position] != char:
                    passed_.append(word)
            return self.notGreen(greenList[1:],passed_)

    def yellow(self, yellowList,yellowset,allowed):

        if yellowset == "":
            return allowed
        
        pattern = re.compile('[{0}]'.format(yellowset))
        passed_ = []
        for word in allowed:
            if self.isMatched(word, pattern):
                passed_.append(word)
        passed_ = self.notGreen(yellowList,passed_)
        return passed_

    #---- WordleWord Test

    def compareWord(self,guess,word, tup = False):
        gray_ = ""
        yellow_ = ""
        yellowList_ = set()
        greenList_ = set()
        
        for i in range(0,5):
            if guess[i] == word[i]:
                greenList_.add((guess[i],i))
            else:
                if guess[i] in word:
                    if guess[i] not in yellow_:
                        yellow_ += guess[i]
                    yellowList_.add((guess[i],i))
                if guess[i] not in word and guess[i] not in gray_:
                    gray_ += guess[i]
        
        if tup:
            return (gray_, tuple(yellowList_), tuple(greenList_))
        else:
            return {    "gray"      : gray_ ,
                    "yellow"    : yellow_,
                    "yellowList" : yellowList_,
                    "greenList" : greenList_ }

    
    def bestGuess(self,allowed,chance):
        if len(allowed) == 1:
            return allowed.pop()
        if chance > 3:
            try:
                guess = random.choices(allowed, weights = [self._allowedWordEntropy[e]  for e in allowed])[0]
                return guess
            except ValueError:
                guess = random.choice(allowed)
        elif chance > 0:
            self.updateProba(allowed)
            try:
                guess = random.choices(allowed, weights = [self._allowedWordEntropy[e]  for e in allowed])[0]
            except ValueError:
                guess = random.choice(allowed)
            return guess
        else:
            guess = random.choice(allowed)
            return guess
        

    def reduce(self, guess_word,allowed):
        meta = self.compareWord(guess_word,self._wordleWord)
        allowed = self.green(list(meta['greenList']),allowed)
        allowed = self.gray(meta['gray'],allowed)
        allowed = self.yellow(list(meta["yellowList"]),meta['yellow'],allowed)
        #print(len(allowed))
        return allowed

    # ---- Remembering Old
    def calEntropy(self,wordprob):
        entropy = {} 
        for key,value in wordprob.items():
            entropy.update({key : value * math.log2(1/value)})
        return entropy

    def letRemember(self):
        try:
            with open("word_frequency.txt","a") as fh:
                fh.write(self._wordleWord + "\n")

        except AttributeError:
            pass

    def updateProba(self,allowed):
        word_frequency = self.readfiles("word_frequency.txt")
        total_count = 23.15 #len(word_frequency)
        word_frequency = Counter(word_frequency)

        #sigmoid = lambda x: 1/(1 + math.exp(-x))
        #prob = lambda x: x if x > 0.5 else 0
        wordprob = {key: value/total_count for key, value in word_frequency.items()}

        for key in allowed:
            if key in wordprob:
                self._allowedWordEntropy[key] *= wordprob[key]
            else:
                self._allowedWordEntropy[key] *= 0.001
        return wordprob
    
    # ----- Run

    def run(self,chance = 6, train = False):
        self._wordleWord = random.choice(self._answers)
        #print(self._wordleWord.upper())

        allowed = self._passed
        while len(allowed) != 0:
            if chance == 0:
                if train:
                    self.letRemember()
                return 0, 0
                re
            chance -= 1
            guess_word = self.bestGuess(allowed,chance)
            #print(guess_word.upper(), end=' ')
            if guess_word == self._wordleWord:
                return 1, chance
            allowed = self.reduce(guess_word, allowed)


In [31]:

gm = Wordle()


In [38]:
mn,mx,avg,count = 100,0,[],0
for i in range(10):
    count += 1
    run = 200
    passed = 0
    chance = 0
    for i in range(run):
        p , c = gm.run()
        if p:
            passed += 1
            chance += (6 - c)
    accuracy = round(passed/run * 100,2)
    if mn > accuracy:
        mn = accuracy
    if mx < accuracy:
        mx = accuracy
    avg.append(accuracy)
    print("{5}. Accuracy : {0}% , Average Chance : {1}, Min Accuracy: {2}, Max Accuracy: {3} , Mean Accuracy: {4} ".format(accuracy,chance/run,mn,mx,sum(avg)/count,count))

1. Accuracy : 94.5% , Average Chance : 4.025, Min Accuracy: 94.5, Max Accuracy: 94.5 , Mean Accuracy: 94.5 
2. Accuracy : 92.5% , Average Chance : 3.87, Min Accuracy: 92.5, Max Accuracy: 94.5 , Mean Accuracy: 93.5 
3. Accuracy : 96.0% , Average Chance : 4.095, Min Accuracy: 92.5, Max Accuracy: 96.0 , Mean Accuracy: 94.33333333333333 
4. Accuracy : 94.0% , Average Chance : 4.05, Min Accuracy: 92.5, Max Accuracy: 96.0 , Mean Accuracy: 94.25 
5. Accuracy : 92.0% , Average Chance : 3.985, Min Accuracy: 92.0, Max Accuracy: 96.0 , Mean Accuracy: 93.8 
6. Accuracy : 95.5% , Average Chance : 4.045, Min Accuracy: 92.0, Max Accuracy: 96.0 , Mean Accuracy: 94.08333333333333 
7. Accuracy : 93.5% , Average Chance : 3.995, Min Accuracy: 92.0, Max Accuracy: 96.0 , Mean Accuracy: 94.0 
8. Accuracy : 97.0% , Average Chance : 4.25, Min Accuracy: 92.0, Max Accuracy: 97.0 , Mean Accuracy: 94.375 
9. Accuracy : 95.5% , Average Chance : 4.095, Min Accuracy: 92.0, Max Accuracy: 97.0 , Mean Accuracy: 94.5 
10